In [ ]:
import os
import sys
import random
from random import sample 

import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.models import load_model
from PIL import Image, ImageOps

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Reshape

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

sys.path.insert(0, os.path.abspath(os.path.join("..", "src")))
from utils import read_filenames, prepare_datasets
from processing import find_bounding_boxes, extract_and_resize, process_image, load_image, convert_to_feature_vectors
from data_generator import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
dataset_path = '../archive/images_gz2/images'
files_per_directory = 100000
img_size = 128

# Initialize an empty list to store the images
images = []

# Traverse the directory structure
for root, dirs, files in os.walk(dataset_path):
    
    limit = files_per_directory
    
    for file in files:
        if file.endswith('.jpg'):
            # Construct the full path to the image
            file_path = os.path.join(root, file)
            
            # Open the image
            img = Image.open(file_path)
            
            # Resize the image to 32x32 pixels
            img = img.resize((img_size, img_size))
            
            # Convert the image to a numpy array and normalize the pixel values
            img_array = np.array(img) / 255.0
            
            # Add the image array to the list
            images.append(img_array)
            
            # Adding a limit to images per directory
            limit -= 1
            if limit == 0:
                break

# Convert the list of images to a numpy array
galaxies = np.array(images)

In [ ]:
len(galaxies)

In [ ]:
galaxies[0z]

In [ ]:
galaxies_train, galaxies_val, galaxies_test = prepare_datasets(galaxies, 10000)

In [ ]:
image = galaxies[30]
plt.imshow(image)
plt.show()

In [ ]:
image_bb = find_bounding_boxes(galaxies[30], 30)
plt.imshow(image_bb)
plt.show()

In [ ]:
random_galaxy = galaxies[random.randint(0, len(galaxies_train) - 1)]

image = cv.imread(random_galaxy)
plt.subplot(1, 2, 1)
plt.xlabel('Before processing')
plt.imshow(image)

processed_image = process_image(random_galaxy, 20, (80, 80))
plt.subplot(1, 2, 2)
plt.xlabel('After processing')
plt.imshow(processed_image)
plt.show()

In [ ]:
batch_size = 32
img_size = 80

train_gen = ImageDataGenerator(galaxies_train, batch_size, img_size)
val_gen = ImageDataGenerator(galaxies_val, batch_size, img_size)

In [ ]:
encoder = Sequential([
    Flatten(input_shape=[80, 80]),
    Dense(1000, activation="relu"),
    Dense(800, activation="relu"),
    Dense(600, activation="relu"),
    Dense(400, activation="relu"),
    Dense(200, activation="relu"),
    Dense(100, activation="relu"),
    Dense(50, activation="relu")
])

# Define the decoder
decoder = Sequential([
    Dense(100, input_shape=[50], activation="relu"),
    Dense(200, activation="relu"),
    Dense(400, activation="relu"),
    Dense(600, activation="relu"),
    Dense(800, activation="relu"),
    Dense(1000, activation="relu"),
    Dense(80*80, activation="sigmoid"),
    Reshape([80, 80])
])

autoencoder = Sequential([encoder, decoder])
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

for epoch in range(10):
    print(f"Epoch {epoch + 1}/{10}")

    autoencoder.fit(
        train_gen,
        epochs=1,
        validation_data=val_gen,
    )

    train_gen.on_epoch_end()
    val_gen.on_epoch_end()

    print()

In [ ]:
processed_sample_val = []

for galaxy in galaxies_val[:100]:
    image = process_image(galaxy, 30, target_size=(80, 80))
    processed_sample_val.append(image)

processed_sample_val = np.array(processed_sample_val)

In [ ]:
predicitions = autoencoder.predict(processed_sample_val[:100])

In [ ]:
index = random.randint(0, len(predicitions) - 1)

plt.subplot(1, 2, 1)
plt.title("Before")
plt.imshow(processed_sample_val[index])

plt.subplot(1, 2, 2)
plt.title("After Auto Encoder")
plt.imshow(predicitions[index])
plt.show()

In [ ]:
model = VGG16(include_top=False, input_shape=(80, 80, 3)) # VGG16 accepts only 3 inputs channels
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

In [ ]:
features = convert_to_feature_vectors(galaxies_train, autoencoder, model, 10000)
features.shape

In [ ]:
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [ ]:
pca = PCA(n_components=700, random_state=21)
pca.fit(features)

In [ ]:
plt.figure(figsize=(12, 6))

xi = np.arange(0, 700, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 701, step=50)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 1, '95% cut-off threshold', color = 'red', fontsize=16)

plt.show()

In [ ]:
n_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.95)

In [ ]:
features_reduced = pca.transform(features)

In [ ]:
def count_wcss_scores(X, k_max):
    scores = []
    for k in range(1, k_max+1):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(X)
        wcss = kmeans.score(X) * -1 # score returns -WCSS
        scores.append(wcss)
    return scores

In [ ]:
wcss_vec = count_wcss_scores(x, 10)
x_ticks = list(range(1, len(wcss_vec) + 1))
plt.plot(x_ticks, wcss_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('Within-cluster sum of squares')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
def count_clustering_scores(X, cluster_num, model, score_fun):
    if isinstance(cluster_num, int):
        cluster_num_iter = [cluster_num]
    else:
        cluster_num_iter = cluster_num
        
    scores = []    
    for k in cluster_num_iter:
        model_instance = model(n_clusters=k)
        labels = model_instance.fit_predict(X)
        wcss = score_fun(X, labels)
        scores.append(wcss)
    
    if isinstance(cluster_num, int):
        return scores[0]
    else:
        return scores

In [ ]:
cluster_num_seq = range(2, 11) # Niektóre metryki nie działają gdy mamy tylko jeden klaster
silhouette_vec = count_clustering_scores(x, cluster_num_seq, KMeans, silhouette_score)
plt.plot(cluster_num_seq, silhouette_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(x)

In [ ]:
groups = {} # cluster_id : images
for file, cluster in zip(galaxies_train[:len(x)], kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
n = 5
fig, axs = plt.subplots(len(groups), n, figsize=(15, 15))

for i, cluster in enumerate(groups.keys()):
    for j in range(n):
        filename = f'../data/images/{groups[cluster][j+5]}'
        img = plt.imread(filename)
        axs[i, j].imshow(img)
        axs[i, j].axis('off')
        axs[i, j].set_title(f"Cluster {cluster}")
plt.show()

In [ ]:
batch_size = 32
img_size = 80

train_gen = ImageDataGenerator(galaxies_train, batch_size, img_size)
val_gen = ImageDataGenerator(galaxies_val, batch_size, img_size)

In [ ]:
encoder = Sequential([
    Flatten(input_shape=[80, 80]),
    Dense(1000, activation="relu"),
    Dense(800, activation="relu"),
    Dense(600, activation="relu"),
    Dense(400, activation="relu"),
    Dense(200, activation="relu"),
    Dense(100, activation="relu"),
    Dense(50, activation="relu")
])

# Define the decoder
decoder = Sequential([
    Dense(100, input_shape=[50], activation="relu"),
    Dense(200, activation="relu"),
    Dense(400, activation="relu"),
    Dense(600, activation="relu"),
    Dense(800, activation="relu"),
    Dense(1000, activation="relu"),
    Dense(80*80, activation="sigmoid"),
    Reshape([80, 80])
])



# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')([80, 80])
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = Conv2D(128, (3, 3), activation='relu', padding='same')([50])
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder
autoencoder = Model([80, 80], decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

for epoch in range(10):
    print(f"Epoch {epoch + 1}/{10}")

    autoencoder.fit(
        train_gen,
        epochs=1,
        validation_data=val_gen,
    )

    train_gen.on_epoch_end()
    val_gen.on_epoch_end()

    print()

In [ ]:
processed_sample_val = []

for galaxy in galaxies_val[:100]:
    image = process_image(galaxy, 30, target_size=(80, 80))
    processed_sample_val.append(image)

processed_sample_val = np.array(processed_sample_val)

In [ ]:
predicitions = autoencoder.predict(processed_sample_val[:100])

In [ ]:
index = random.randint(0, len(predicitions) - 1)

plt.subplot(1, 2, 1)
plt.title("Before")
plt.imshow(processed_sample_val[index])

plt.subplot(1, 2, 2)
plt.title("After Auto Encoder")
plt.imshow(predicitions[index])
plt.show()

In [ ]:
model = VGG16(include_top=False, input_shape=(80, 80, 3)) # VGG16 accepts only 3 inputs channels
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

In [ ]:
features = convert_to_feature_vectors(galaxies_train, autoencoder, model, 10000)
features.shape

In [ ]:
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [ ]:
pca = PCA(n_components=700, random_state=21)
pca.fit(features)

In [ ]:
plt.figure(figsize=(12, 6))

xi = np.arange(0, 700, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 701, step=50)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 1, '95% cut-off threshold', color = 'red', fontsize=16)

plt.show()

In [ ]:
n_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.95)

In [ ]:
features_reduced = pca.transform(features)

In [ ]:
def count_wcss_scores(X, k_max):
    scores = []
    for k in range(1, k_max+1):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(X)
        wcss = kmeans.score(X) * -1 # score returns -WCSS
        scores.append(wcss)
    return scores

In [ ]:
wcss_vec = count_wcss_scores(x, 10)
x_ticks = list(range(1, len(wcss_vec) + 1))
plt.plot(x_ticks, wcss_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('Within-cluster sum of squares')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
def count_clustering_scores(X, cluster_num, model, score_fun):
    if isinstance(cluster_num, int):
        cluster_num_iter = [cluster_num]
    else:
        cluster_num_iter = cluster_num
        
    scores = []    
    for k in cluster_num_iter:
        model_instance = model(n_clusters=k)
        labels = model_instance.fit_predict(X)
        wcss = score_fun(X, labels)
        scores.append(wcss)
    
    if isinstance(cluster_num, int):
        return scores[0]
    else:
        return scores

In [ ]:
cluster_num_seq = range(2, 11) # Niektóre metryki nie działają gdy mamy tylko jeden klaster
silhouette_vec = count_clustering_scores(x, cluster_num_seq, KMeans, silhouette_score)
plt.plot(cluster_num_seq, silhouette_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(x)

In [ ]:
groups = {} # cluster_id : images
for file, cluster in zip(galaxies_train[:len(x)], kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
n = 5
fig, axs = plt.subplots(len(groups), n, figsize=(15, 15))

for i, cluster in enumerate(groups.keys()):
    for j in range(n):
        filename = f'../data/images/{groups[cluster][j+5]}'
        img = plt.imread(filename)
        axs[i, j].imshow(img)
        axs[i, j].axis('off')
        axs[i, j].set_title(f"Cluster {cluster}")
plt.show()

In [ ]:
batch_size = 32
img_size = 80

train_gen = ImageDataGenerator(galaxies_train, batch_size, img_size)
val_gen = ImageDataGenerator(galaxies_val, batch_size, img_size)

encoder = Sequential([
    Flatten(input_shape=[80, 80]),
    Dense(1000, activation="relu"),
    Dense(800, activation="relu"),
    Dense(600, activation="relu"),
    Dense(400, activation="relu"),
    Dense(200, activation="relu"),
    Dense(100, activation="relu"),
    Dense(50, activation="relu")
])

# Define the decoder
decoder = Sequential([
    Dense(100, input_shape=[50], activation="relu"),
    Dense(200, activation="relu"),
    Dense(400, activation="relu"),
    Dense(600, activation="relu"),
    Dense(800, activation="relu"),
    Dense(1000, activation="relu"),
    Dense(80*80, activation="sigmoid"),
    Reshape([80, 80])
])

autoencoder = Sequential([encoder, decoder])
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

for epoch in range(10):
    print(f"Epoch {epoch + 1}/{10}")

    autoencoder.fit(
        train_gen,
        epochs=1,
        validation_data=val_gen,
    )

    train_gen.on_epoch_end()
    val_gen.on_epoch_end()

    print()

processed_sample_val = []

for galaxy in galaxies_val[:100]:
    image = process_image(galaxy, 30, target_size=(80, 80))
    processed_sample_val.append(image)

processed_sample_val = np.array(processed_sample_val)

predicitions = autoencoder.predict(processed_sample_val[:100])

index = random.randint(0, len(predicitions) - 1)

plt.subplot(1, 2, 1)
plt.title("Before")
plt.imshow(processed_sample_val[index])

plt.subplot(1, 2, 2)
plt.title("After Auto Encoder")
plt.imshow(predicitions[index])
plt.show()

model = VGG16(include_top=False, input_shape=(80, 80, 3)) # VGG16 accepts only 3 inputs channels
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

features = convert_to_feature_vectors(galaxies_train, autoencoder, model, 10000)
features.shape

scaler = StandardScaler()
features = scaler.fit_transform(features)

pca = PCA(n_components=700, random_state=21)
pca.fit(features)

plt.figure(figsize=(12, 6))

xi = np.arange(0, 700, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 701, step=50)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 1, '95% cut-off threshold', color = 'red', fontsize=16)

plt.show()

n_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.95)

features_reduced = pca.transform(features)

def count_wcss_scores(X, k_max):
    scores = []
    for k in range(1, k_max+1):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(X)
        wcss = kmeans.score(X) * -1 # score returns -WCSS
        scores.append(wcss)
    return scores

wcss_vec = count_wcss_scores(x, 10)
x_ticks = list(range(1, len(wcss_vec) + 1))
plt.plot(x_ticks, wcss_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('Within-cluster sum of squares')
plt.title('The Elbow Method showing the optimal k')
plt.show()

def count_clustering_scores(X, cluster_num, model, score_fun):
    if isinstance(cluster_num, int):
        cluster_num_iter = [cluster_num]
    else:
        cluster_num_iter = cluster_num
        
    scores = []    
    for k in cluster_num_iter:
        model_instance = model(n_clusters=k)
        labels = model_instance.fit_predict(X)
        wcss = score_fun(X, labels)
        scores.append(wcss)
    
    if isinstance(cluster_num, int):
        return scores[0]
    else:
        return scores

cluster_num_seq = range(2, 11) # Niektóre metryki nie działają gdy mamy tylko jeden klaster
silhouette_vec = count_clustering_scores(x, cluster_num_seq, KMeans, silhouette_score)
plt.plot(cluster_num_seq, silhouette_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.show()

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(x)

groups = {} # cluster_id : images
for file, cluster in zip(galaxies_train[:len(x)], kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

n = 5
fig, axs = plt.subplots(len(groups), n, figsize=(15, 15))

for i, cluster in enumerate(groups.keys()):
    for j in range(n):
        filename = f'../data/images/{groups[cluster][j+5]}'
        img = plt.imread(filename)
        axs[i, j].imshow(img)
        axs[i, j].axis('off')
        axs[i, j].set_title(f"Cluster {cluster}")
plt.show()